# Segmentation 3D

## Import modules

In [2]:
import openalea.phenomenal.display as phm_display
import openalea.phenomenal.data as phm_data
import openalea.phenomenal.display.notebook as phm_display_notebook
import openalea.phenomenal.object as phm_obj
import openalea.phenomenal.multi_view_reconstruction as phm_mvr
import openalea.phenomenal.segmentation as phm_seg

## Load data - Voxel Grid, Binaries image and calibration

In [3]:
plant_number = 2
voxels_size = 16

bin_images = phm_data.bin_images("data/plant_2")
calibrations = phm_data.calibrations("data/plant_2")
voxel_grid = phm_data.voxel_grid("data", plant_number=2, voxels_size=voxels_size)

# Display voxel grid
phm_display_notebook.show_voxel_grid(voxel_grid, size=1)

## Skeletonization

### Compute Graph, source node and the skeleton

In [4]:
# 1. Compute graph of neighborhood
graph = phm_seg.graph_from_voxel_grid(voxel_grid, connect_all_point=True)
# 2. Find source node 
src_node = tuple(phm_seg.find_base_stem_position(graph.nodes(), 
                                                 voxel_grid.voxels_size, 
                                                 neighbor_size=45))
# 3. Compute Skeleton
voxel_skeleton = phm_seg.skeletonize(voxel_grid, graph, src_node)
# 4. Display result
phm_display_notebook.show_skeleton(voxel_skeleton, with_voxel=True, size=1)

### Remove not enouth visible segment

In [5]:
# Select images
image_projection = list()
for angle in bin_images["side"]:
    projection = calibrations["side"].get_projection(angle)
    image_projection.append((bin_images["side"][angle], projection))

voxel_skeleton_denoised = phm_seg.segment_reduction(
    voxel_skeleton, image_projection, required_visible=5, nb_min_pixel=400)

# Display result
phm_display_notebook.show_skeleton(voxel_skeleton_denoised, size=1)

## Maize Segmentation

Voxel skeleton are segmented in several label "stem", "mature_leaf", "growing_leaf" and "unknow"

In [6]:
vms = phm_seg.maize_segmentation(voxel_skeleton_denoised, graph)

# Write
vms.write_to_json_gz('voxel_maize_segmentation.json')

# Read
vms = phm_obj.VoxelSegmentation.read_from_json_gz('voxel_maize_segmentation.json')

# Display
phm_display_notebook.show_segmentation(vms, size=1)

## Maize Analysis

In [7]:
vmsi = phm_seg.maize_analysis(vms)

# write
vmsi.write_to_json_gz('voxel_maize_segmentation_info.json.gz')

# read
vmsi = phm_obj.VoxelSegmentation.read_from_json_gz('voxel_maize_segmentation_info.json.gz')

# Display result
phm_display_notebook.show_segmentation(vmsi, size=1)



In [8]:
import pandas

df = pandas.DataFrame([vo.info for vo in vmsi.voxel_organs]  + [vmsi.info])
df

,pm_label,pm_sub_label,pm_voxels_volume,pm_position_tip,pm_position_base,pm_z_tip,pm_z_base,pm_length,pm_width_max,pm_width_mean,...,pm_vector_mean,pm_azimuth_angle,pm_inclination_angle,pm_full_length,pm_length_with_speudo_stem,pm_insertion_angle_vector,pm_insertion_angle,pm_length_speudo_stem,pm_leaf_number,pm_number_of_leaf
0,unknown,NaN,233472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,stem,NaN,1040384,"[8.0, -24.0, 536.0]","[8.0, -24.0, -696.0]",536.0,-696.0,1232.000000,35.777088,22.574245,...,"[0.0, 0.0, 624.0]",0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,growing_leaf,NaN,4497408,"[24.0, 24.0, 1256.0]","[8.0, -8.0, 984.0]",1256.0,984.0,290.340230,71.554175,54.695777,...,"[3.764705882352941, 4.705882352941177, 144.0]",51.340192,0.513619,1976.967647,728.967647,"[0.0, 0.0, 40.0]",0.000000,438.627417,13.0,NaN
3,growing_leaf,NaN,4313088,"[-40.0, -56.0, 1192.0]","[8.0, -24.0, 888.0]",1192.0,888.0,345.765856,132.905982,62.660490,...,"[-36.21052631578947, -35.36842105263158, 160.0]",-135.674037,0.847678,1929.765856,681.765856,"[-16.0, -16.0, 48.0]",25.239402,336.000000,12.0,NaN
4,growing_leaf,NaN,3264512,"[248.0, 440.0, 824.0]","[8.0, -24.0, 536.0]",824.0,536.0,853.116466,131.939380,77.251913,...,"[109.26829268292683, 205.26829268292684, 263.0...",61.972761,1.596087,2085.116466,837.116466,"[11.2, 11.2, 88.0]",10.203464,-16.000000,10.0,NaN
5,growing_leaf,NaN,4759552,"[-216.0, -424.0, 952.0]","[8.0, -24.0, 728.0]",952.0,728.0,635.743883,164.730082,89.794619,...,"[-125.03703703703704, -198.5185185185185, 187....",-122.204805,2.482344,2059.743883,811.743883,"[-20.571428571428573, -57.142857142857146, 64.0]",43.499626,176.000000,11.0,NaN
6,mature_leaf,NaN,3047424,"[-520.0, -392.0, 584.0]","[-24.0, -40.0, 440.0]",584.0,440.0,802.250843,125.984126,84.079929,...,"[-288.0, -117.77777777777777, 166.22222222222223]",-157.757862,2.185531,1956.591073,NaN,"[-80.0, 0.0, 80.0]",45.000000,NaN,9.0,NaN
7,mature_leaf,NaN,3235840,"[616.0, 376.0, 312.0]","[24.0, -24.0, 264.0]",312.0,264.0,900.240534,139.484766,85.699344,...,"[304.0, 188.97297297297297, 150.48648648648648]",31.865978,2.341954,1866.867951,NaN,"[80.0, 16.0, 80.0]",45.561759,NaN,8.0,NaN
8,mature_leaf,NaN,3084288,"[-648.0, 136.0, 8.0]","[-8.0, -24.0, 104.0]",8.0,104.0,950.646824,122.898332,84.517146,...,"[-338.6666666666667, 136.38095238095238, 99.04...",158.065371,2.315972,1757.274241,NaN,"[-88.0, 36.8, 86.4]",47.829537,NaN,7.0,NaN
9,mature_leaf,NaN,2580480,"[408.0, -72.0, -200.0]","[40.0, -40.0, -8.0]",-200.0,-8.0,766.785192,148.377896,81.893836,...,"[271.6190476190476, -35.42857142857143, 34.285...",-7.431408,2.761813,1473.125422,NaN,"[88.0, -12.8, 60.8]",55.639027,NaN,6.0,NaN
